In [217]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [219]:
print(os.getcwd())  # Muestra la ruta actual

C:\Users\57302\Documents\Analisis de datos\Proyecto_ integrador_sql_python


In [7]:
os.chdir("C:/Users/57302/Documents/Analisis de datos/Proyecto_ integrador_sql_python")
print("Nueva ruta:", os.getcwd())  # Verifica el cambio

Nueva ruta: C:\Users\57302\Documents\Analisis de datos\Proyecto_ integrador_sql_python


In [138]:
# Cargar los archivos CSV
df_city = pd.read_csv("etl_proyecto_mundial/datos_csv/city.csv")
df_country = pd.read_csv("etl_proyecto_mundial/datos_csv/country.csv")
df_countrylanguage = pd.read_csv("etl_proyecto_mundial/datos_csv/countrylanguage.csv")

#### 🔍 Exploración de City

In [11]:
df_city

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
...,...,...,...,...,...
4074,4075,Khan Yunis,PSE,Khan Yunis,123175
4075,4076,Hebron,PSE,Hebron,119401
4076,4077,Jabaliya,PSE,North Gaza,113901
4077,4078,Nablus,PSE,Nablus,100231


In [13]:
print("📊 Información General de City\n")
df_city.info()

📊 Información General de City

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4079 entries, 0 to 4078
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           4079 non-null   int64 
 1   Name         4079 non-null   object
 2   CountryCode  4079 non-null   object
 3   District     4075 non-null   object
 4   Population   4079 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 159.5+ KB


In [15]:
# Valores unicos de id 
len(df_city.ID.unique())

4079

Se observa que no hay valores repetidos del ID city, ya que la cantidad de valores únicos coincide con la cantidad total de datos.

In [17]:
# Valores nulos en cada DataFrame
def detectar_nulos(df, nombre):
    nulos = df.isnull().sum()
    porcentaje = (df.isnull().mean() * 100).round(2) # Convertir a porcentaje y redondear a 2 decimales
    df_nulos = pd.DataFrame({"Valores Nulos": nulos, "Porcentaje (%)": porcentaje})
    df_nulos = df_nulos[df_nulos["Valores Nulos"] > 0] # Filtrar solo las columnas que tienen valores nulos
    print(f"\n🔍 Valores nulos en {nombre}:")
    display(df_nulos)

In [19]:
detectar_nulos(df_city, "City")


🔍 Valores nulos en City:


,Valores Nulos,Porcentaje (%)
District,4,0.1


##### -📝 Manejo de Valores Nulos en el df City, columna `District`

In [21]:
# Filtrar filas con valores nulos en la columna District y mostrar el resto de la información
print("\n🔍 Filas con valores nulos en la columna 'District' en City:")
df_nulos_district = df_city[df_city["District"].isnull()]
df_nulos_district


🔍 Filas con valores nulos en la columna 'District' en City:


,ID,Name,CountryCode,District,Population
3284,3285,Taiping,TWN,NaN,165524
3292,3293,Taliao,TWN,NaN,115897
3293,3294,Kueishan,TWN,NaN,112195
3562,3563,Ciudad Losada,VEN,NaN,134501


Se realizó una **unión** con la tabla `country` para obtener más contexto sobre cada ciudad, incluyendo el **nombre del país**, la **región** y el **área superficial**; con el fin de  analizar mejor la información y facilitar la búsqueda de los distritos correctos.

In [25]:
df_nulos_district = df_nulos_district.merge(
    df_country[["Code", "Name", "Region", "SurfaceArea"]].rename(columns={"Name": "NamePais"}), 
    left_on="CountryCode",  # Columna en df_nulos_district
    right_on="Code",        # Columna en df_country
    how="left"
)
df_nulos_district

,ID,Name,CountryCode,District,Population,Code,NamePais,Region,SurfaceArea
0,3285,Taiping,TWN,NaN,165524,TWN,Taiwan,Eastern Asia,36188.0
1,3293,Taliao,TWN,NaN,115897,TWN,Taiwan,Eastern Asia,36188.0
2,3294,Kueishan,TWN,NaN,112195,TWN,Taiwan,Eastern Asia,36188.0
3,3563,Ciudad Losada,VEN,NaN,134501,VEN,Venezuela,South America,912050.0


###### 🌍 Búsqueda de información externa
No obstante, debido a la ausencia de datos en la fuente original, consulté fuentes externas para determinar los distritos correspondientes a cada ciudad con valores nulos


###### ✅ Corrección y actualización de valores en VS Code  
Se asignaron manualmente los valores correctos a la columna `"District"` utilizando un diccionario de mapeo y la función `.map()`.

Estos cambios fueron implementados en el **archivo de transformación dentro del pipeline ETL en VS Code**, asegurando que la corrección de los valores nulos forme parte del **proceso automatizado**. Finalmente, los datos transformados fueron guardados en el archivo CSV actualizado, garantizando que el dataset se mantenga limpio y listo para su análisis.  

#### 🔍 Exploración de Country

In [38]:
df_country

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav Koštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM


In [40]:
print("📊 Información General de Cpuntry\n")
df_country.info()

📊 Información General de Cpuntry

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Code            239 non-null    object 
 1   Name            239 non-null    object 
 2   Continent       239 non-null    object 
 3   Region          239 non-null    object 
 4   SurfaceArea     239 non-null    float64
 5   IndepYear       192 non-null    float64
 6   Population      239 non-null    int64  
 7   LifeExpectancy  222 non-null    float64
 8   GNP             239 non-null    float64
 9   GNPOld          178 non-null    float64
 10  LocalName       239 non-null    object 
 11  GovernmentForm  239 non-null    object 
 12  HeadOfState     236 non-null    object 
 13  Capital         232 non-null    float64
 14  Code2           238 non-null    object 
dtypes: float64(6), int64(1), object(8)
memory usage: 28.1+ KB


In [48]:
# Contar valores unicos de Code
len(df_country.Code.unique())

239

In [73]:
# Contar valores únicos en 'Name'
unique_names = df_country["Name"].nunique()
total_records = len(df_country)

print(f"Total de registros en country: {total_records}")
print(f"Nombres únicos en country: {unique_names}")

if unique_names < total_records:
    print("⚠️ Hay nombres duplicados en la tabla Country.")
    display(df_country[df_country.duplicated(subset=["Name"], keep=False)].sort_values("Name"))
else:
    print("✅ No hay nombres de países duplicados en la tabla Country.")

Total de registros en country: 239
Nombres únicos en country: 239
✅ No hay nombres duplicados en la tabla Country.


###### 📌 Validación de Valores Únicos en la Tabla `Country`

Para asegurar la integridad de los datos en la tabla `Country`, se realizó la validación de los valores únicos en las columnas `Code` y `Name`:

- **Código del país (`Code`)**: Se verificó que cada país tiene un código único, sin repeticiones.
- **Nombre del país (`Name`)**: También se comprobó que los nombres de los países no se repiten, garantizando que no haya registros duplicados.


In [42]:
detectar_nulos(df_country, "Country")


🔍 Valores nulos en Country:


,Valores Nulos,Porcentaje (%)
IndepYear,47,19.67
LifeExpectancy,17,7.11
GNPOld,61,25.52
HeadOfState,3,1.26
Capital,7,2.93
Code2,1,0.42


In [53]:
# Filtrar filas con valores nulos en la columna IndepYear 
print("\n🔍 Filas con valores nulos en la columna 'IndepYear' en Country:")
df_nulos_indepyear = df_country[df_country["IndepYear"].isnull()]
df_nulos_indepyear


🔍 Filas con valores nulos en la columna 'IndepYear' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
6,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.0,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33.0,AN
10,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,68000,75.1,334.0,NaN,Amerika Samoa,US Territory,George W. Bush,54.0,AS
11,ATA,Antarctica,Antarctica,Antarctica,13120000.0,NaN,0,NaN,0.0,NaN,–,Co-administrated,NaN,NaN,AQ
12,ATF,French Southern territories,Antarctica,Antarctica,7780.0,NaN,0,NaN,0.0,NaN,Terres australes françaises,Nonmetropolitan Territory of France,Jacques Chirac,NaN,TF
28,BMU,Bermuda,North America,North America,53.0,NaN,65000,76.9,2328.0,2190.0,Bermuda,Dependent Territory of the UK,Elisabeth II,191.0,BM
34,BVT,Bouvet Island,Antarctica,Antarctica,59.0,NaN,0,NaN,0.0,NaN,Bouvetøya,Dependent Territory of Norway,Harald V,NaN,BV
38,CCK,Cocos (Keeling) Islands,Oceania,Australia and New Zealand,14.0,NaN,600,NaN,0.0,NaN,Cocos (Keeling) Islands,Territory of Australia,Elisabeth II,2317.0,CC
46,COK,Cook Islands,Oceania,Polynesia,236.0,NaN,20000,71.1,100.0,NaN,The Cook Islands,Nonmetropolitan Territory of New Zealand,Elisabeth II,583.0,CK


##### -📝 Manejo de Valores Nulos en el df Country, columna `IndepYear`

Se identificaron 47 valores nulos en la columna `IndepYear`, lo que indica que estos países o territorios no tienen un año de independencia registrado. Esto se debe a diferentes razones, como:

1. **Dependencias o territorios de otros países**, como Groenlandia (Dinamarca) o Puerto Rico (EE.UU.).
2. **Regiones administradas por organismos internacionales**, como Palestina o el Sahara Occidental.
3. **Territorios deshabitados o sin gobierno propio**, como la Antártida y Bouvet Island.

Estos valores nulos no representan un error en los datos, sino que reflejan la realidad política y administrativa de cada país o territorio.

Para mantener la integridad de los datos y evitar problemas en cálculos numéricos, no se modificó la columna "IndepYear". En su lugar, se creó una nueva columna "EsIndependizado", donde:

✅ "Sí" → País con año de independencia registrado.
❌ "No aplica" → País sin independencia registrada.

Este enfoque permite análisis más precisos sin alterar los datos originales

In [90]:
# Filtrar filas con valores nulos en la columna LifeExpectancy 
print("\n🔍 Filas con valores nulos en la columna 'LifeExpectancy' en Country:")
df_nulos_lifeexpctancy = df_country[df_country["LifeExpectancy"].isnull()]
df_nulos_lifeexpctancy


🔍 Filas con valores nulos en la columna 'LifeExpectancy' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
11,ATA,Antarctica,Antarctica,Antarctica,13120000.0,NaN,0,NaN,0.0,NaN,–,Co-administrated,NaN,NaN,AQ
12,ATF,French Southern territories,Antarctica,Antarctica,7780.0,NaN,0,NaN,0.0,NaN,Terres australes françaises,Nonmetropolitan Territory of France,Jacques Chirac,NaN,TF
34,BVT,Bouvet Island,Antarctica,Antarctica,59.0,NaN,0,NaN,0.0,NaN,Bouvetøya,Dependent Territory of Norway,Harald V,NaN,BV
38,CCK,Cocos (Keeling) Islands,Oceania,Australia and New Zealand,14.0,NaN,600,NaN,0.0,NaN,Cocos (Keeling) Islands,Territory of Australia,Elisabeth II,2317.0,CC
52,CXR,Christmas Island,Oceania,Australia and New Zealand,135.0,NaN,2500,NaN,0.0,NaN,Christmas Island,Territory of Australia,Elisabeth II,1791.0,CX
71,FLK,Falkland Islands,South America,South America,12173.0,NaN,2000,NaN,0.0,NaN,Falkland Islands,Dependent Territory of the UK,Elisabeth II,763.0,FK
93,HMD,Heard Island and McDonald Islands,Antarctica,Antarctica,359.0,NaN,0,NaN,0.0,NaN,Heard and McDonald Islands,Territory of Australia,Elisabeth II,NaN,HM
100,IOT,British Indian Ocean Territory,Africa,Eastern Africa,78.0,NaN,0,NaN,0.0,NaN,British Indian Ocean Territory,Dependent Territory of the UK,Elisabeth II,NaN,IO
154,NFK,Norfolk Island,Oceania,Australia and New Zealand,36.0,NaN,2000,NaN,0.0,NaN,Norfolk Island,Territory of Australia,Elisabeth II,2806.0,NF
157,NIU,Niue,Oceania,Polynesia,260.0,NaN,2000,NaN,0.0,NaN,Niue,Nonmetropolitan Territory of New Zealand,Elisabeth II,2805.0,NU


In [100]:
# Filtrar registros donde LifeExpectancy es NaN y Population es 0
df_nulos_life_expectancy = df_country[(df_country["LifeExpectancy"].isna()) & (df_country["Population"] == 0)]
df_nulos_life_expectancy

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
11,ATA,Antarctica,Antarctica,Antarctica,13120000.0,NaN,0,NaN,0.0,NaN,–,Co-administrated,NaN,NaN,AQ
12,ATF,French Southern territories,Antarctica,Antarctica,7780.0,NaN,0,NaN,0.0,NaN,Terres australes françaises,Nonmetropolitan Territory of France,Jacques Chirac,NaN,TF
34,BVT,Bouvet Island,Antarctica,Antarctica,59.0,NaN,0,NaN,0.0,NaN,Bouvetøya,Dependent Territory of Norway,Harald V,NaN,BV
93,HMD,Heard Island and McDonald Islands,Antarctica,Antarctica,359.0,NaN,0,NaN,0.0,NaN,Heard and McDonald Islands,Territory of Australia,Elisabeth II,NaN,HM
100,IOT,British Indian Ocean Territory,Africa,Eastern Africa,78.0,NaN,0,NaN,0.0,NaN,British Indian Ocean Territory,Dependent Territory of the UK,Elisabeth II,NaN,IO
187,SGS,South Georgia and the South Sandwich Islands,Antarctica,Antarctica,3903.0,NaN,0,NaN,0.0,NaN,South Georgia and the South Sandwich Islands,Dependent Territory of the UK,Elisabeth II,NaN,GS
221,UMI,United States Minor Outlying Islands,Oceania,Micronesia/Caribbean,16.0,NaN,0,NaN,0.0,NaN,United States Minor Outlying Islands,Dependent Territory of the US,George W. Bush,NaN,UM


In [102]:
len(df_nulos_life_expectancy)

7

##### 📝 Manejo de Valores Nulos en el df Country, columna `LifeExpectancy` 

Al analizar la columna `LifeExpectancy`, se encontró 17 registros con valores nulos. Estos casos corresponden principalmente a:

- **Territorios deshabitados o con población extremadamente baja** (Ej. Antártida, Bouvet Island).
- **Territorios dependientes de otros países** (Ej. Cocos (Keeling) Islands, Falkland Islands (Islas Malvinas)).
- **Micropaíses con datos limitados** (Ej. Niue, Holy See (Vatican City State)).

###### 🛠️ **Decisión: Mantener los valores `NaN`**  
Modificar estos valores podría introducir datos falsos o afectar la integridad del dataset. Por ello, he decidido **mantener los `NaN`** en `LifeExpectancy`, asegurando que cualquier análisis posterior pueda manejar estos valores de manera adecuada.

In [96]:
# Filtrar filas con valores nulos en la columna GNPold 
print("\n🔍 Filas con valores nulos en la columna 'GNPOld' en Country:")
df_nulos_gnpold = df_country[df_country["GNPOld"].isnull()]
df_nulos_gnpold


🔍 Filas con valores nulos en la columna 'GNPOld' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
5,AND,Andorra,Europe,Southern Europe,468.0,1278.0,78000,83.5,1630.0,NaN,Andorra,Parliamentary Coprincipality,NaN,55.0,AD
6,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.0,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33.0,AN
10,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,68000,75.1,334.0,NaN,Amerika Samoa,US Territory,George W. Bush,54.0,AS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,VAT,Holy See (Vatican City State),Europe,Southern Europe,0.4,1929.0,1000,NaN,9.0,NaN,Santa Sede/Città del Vaticano,Independent Church State,Johannes Paavali II,3538.0,VA
226,VCT,Saint Vincent and the Grenadines,North America,Caribbean,388.0,1979.0,114000,72.3,285.0,NaN,Saint Vincent and the Grenadines,Constitutional Monarchy,Elisabeth II,3066.0,VC
229,VIR,"Virgin Islands, U.S.",North America,Caribbean,347.0,NaN,93000,78.1,0.0,NaN,Virgin Islands of the United States,US Territory,George W. Bush,4067.0,VI
232,WLF,Wallis and Futuna,Oceania,Polynesia,200.0,NaN,15000,NaN,0.0,NaN,Wallis-et-Futuna,Nonmetropolitan Territory of France,Jacques Chirac,3536.0,WF


In [172]:
# Consulta para conocer el Jefe de Estado de Colombia, mi país natal
df_country[df_country.Code == 'COL'] [['HeadOfState']]

,HeadOfState
47,Andrés Pastrana Arango


A través de la anterior consulta, se identificó que el jefe de Estado registrado en la base de datos coincide con el periodo de gobierno de un presidente colombiano cuyo mandato estuvo entre **1998 y 2002**.

Este hallazgo sugiere que los valores en la base de datos corresponden a un momento dentro de este intervalo temporal.

##### **- Implicaciones**
Este rango de años nos permite interpretar con mayor precisión las variables presentes en la base de datos, especialmente aquellas relacionadas con el Producto Nacional Bruto (`GNP` y `GNPOld`), cuya diferencia podría representar una variación entre años consecutivos dentro de este periodo.

##### 🔍 Manejo de Valores Nulos en el df Country, columna `GNPOld` 

El df country.csv contiene información sobre el Producto Nacional Bruto (PNB) actual (GNP) y un valor anterior (GNPOld). Se identificó que GNPOld tiene 61 valores nulos, equivalentes al 25.52% del total de registros.

In [116]:
# Explorar datos
pd.options.display.float_format = '{:,.2f}'.format
df_country[['GNP', 'GNPOld']].describe()

,GNP,GNPOld
count,239.00,178.00
mean,"122,823.88","165,534.31"
std,"637,997.58","720,468.91"
min,0.00,157.00
25%,640.00,"2,187.00"
50%,"4,787.00","8,421.00"
75%,"29,944.50","71,145.50"
max,"8,510,700.00","8,110,900.00"


In [120]:
# Correlación entre GNP y GNPOld
correlation = df_country[['GNP', 'GNPOld']].corr().iloc[0, 1]
print(f'Correlación entre GNP y GNPOld: {correlation:.3f}')

Correlación entre GNP y GNPOld: 0.998


Se calculó una correlación de 0.998, lo que indica que GNPOld está altamente relacionado con GNP.

Esto sugiere que GNPOld representa el PNB de un año anterior o de una actualización previa.

###### 🛠️ Decisión: La mediana de GNP se considera un buen estimador para completar los valores faltantes de GNPOld.

In [150]:
# Filtrar filas con valores nulos en la columna HeadOfState 
print("\n🔍 Filas con valores nulos en la columna 'HeadOfState' en Country:")
df_nulos_headofstate = df_country[df_country["HeadOfState"].isnull()]
df_nulos_headofstate


🔍 Filas con valores nulos en la columna 'HeadOfState' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2,EsIndependizado
5,AND,Andorra,Europe,Southern Europe,468.00,"1,278.00",78000,83.50,"1,630.00","4,787.00",Andorra,Parliamentary Coprincipality,NaN,55.00,AD,Sí
11,ATA,Antarctica,Antarctica,Antarctica,"13,120,000.00",NaN,0,NaN,0.00,"4,787.00",–,Co-administrated,NaN,NaN,AQ,No aplica
193,SMR,San Marino,Europe,Southern Europe,61.00,885.00,27000,81.10,510.00,"4,787.00",San Marino,Republic,NaN,"3,171.00",SM,Sí


##### 🛠️ Actualización de Jefes de Estado para 2002

Se evidencia que tres países tenían datos nulos para jefes de estado, pero tras realizar una búsqueda externae, se encontró que **Andorra** y **San Marino** sí tienen jefes de estado en 2002:

- **Andorra**: Tiene dos copríncipes: **Joan-Enric Vives i Sicília** (Obispo de Urgell) y **Jacques Chirac** (Presidente de Francia).
- **San Marino**: Tiene dos Capitanes Regentes, que cambian cada seis meses.

**Antártida**, sin embargo, no tiene jefe de estado debido a su estatus internacional bajo el Tratado Antártico.

In [190]:
# Filtrar filas con valores nulos en la columna Capital 
print("\n🔍 Filas con valores nulos en la columna 'Capital ' en Country:")
df_nulos_capital = df_country[df_country["Capital"].isnull()]
df_nulos_capital


🔍 Filas con valores nulos en la columna 'Capital ' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2,EsIndependizado
11,ATA,Antarctica,Antarctica,Antarctica,"13,120,000.00",NaN,0,NaN,0.00,"4,787.00",–,Co-administrated,NaN,NaN,AQ,No aplica
12,ATF,French Southern territories,Antarctica,Antarctica,"7,780.00",NaN,0,NaN,0.00,"4,787.00",Terres australes françaises,Nonmetropolitan Territory of France,Jacques Chirac,NaN,TF,No aplica
34,BVT,Bouvet Island,Antarctica,Antarctica,59.00,NaN,0,NaN,0.00,"4,787.00",Bouvetøya,Dependent Territory of Norway,Harald V,NaN,BV,No aplica
93,HMD,Heard Island and McDonald Islands,Antarctica,Antarctica,359.00,NaN,0,NaN,0.00,"4,787.00",Heard and McDonald Islands,Territory of Australia,Elisabeth II,NaN,HM,No aplica
100,IOT,British Indian Ocean Territory,Africa,Eastern Africa,78.00,NaN,0,NaN,0.00,"4,787.00",British Indian Ocean Territory,Dependent Territory of the UK,Elisabeth II,NaN,IO,No aplica
187,SGS,South Georgia and the South Sandwich Islands,Antarctica,Antarctica,"3,903.00",NaN,0,NaN,0.00,"4,787.00",South Georgia and the South Sandwich Islands,Dependent Territory of the UK,Elisabeth II,NaN,GS,No aplica
221,UMI,United States Minor Outlying Islands,Oceania,Micronesia/Caribbean,16.00,NaN,0,NaN,0.00,"4,787.00",United States Minor Outlying Islands,Dependent Territory of the US,George W. Bush,NaN,UM,No aplica


In [192]:
# Filtrar filas con valores nulos en la columna Capital 
print("\n🔍 Filas con valores nulos en la columna 'Code2' en Country:")
df_nulos_code2 = df_country[df_country["Code2"].isnull()]
df_nulos_code2


🔍 Filas con valores nulos en la columna 'Code2' en Country:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2,EsIndependizado
151,NAM,Namibia,Africa,Southern Africa,"824,292.00","1,990.00",1726000,42.50,"3,101.00","3,384.00",Namibia,Republic,Sam Nujoma,"2,726.00",NaN,Sí


Según el código ISO 3166-1 Alpha-2 de Namibia es "NA", por lo tanto se completa el campo Code2 con "NA" para mantener la coherencia del dataset

#### 🔍 Exploración de Countrylanguage

In [202]:
df_countrylanguage

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.30
1,ABW,English,F,9.50
2,ABW,Papiamento,F,76.70
3,ABW,Spanish,F,7.40
4,AFG,Balochi,F,0.90
...,...,...,...,...
979,ZMB,Tonga,F,11.00
980,ZWE,English,T,2.20
981,ZWE,Ndebele,F,16.20
982,ZWE,Nyanja,F,2.20


In [208]:
print("📊 Información General de Cpuntry\n")
df_countrylanguage.info()

📊 Información General de Cpuntry

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CountryCode  984 non-null    object 
 1   Language     984 non-null    object 
 2   IsOfficial   984 non-null    object 
 3   Percentage   984 non-null    float64
dtypes: float64(1), object(3)
memory usage: 30.9+ KB


En este caso, se observa que la cantidad de registros en Country (239) y CountryLanguage (984) no coincide, lo cual es esperable, ya que un mismo país puede tener varios idiomas oficiales o hablados.